## Considere a base de dados **Customer Data and Bank Loans**
- https://www.kaggle.com/datasets/daniellopez01/credit-risk
- Formulário para envio da Atividade: https://forms.gle/5jCooRKSFvKDvtGG6

In [45]:
# !pip install kagglehub

#!pip install pandas

In [46]:
#import kagglehub

    #   Download latest version
#path = kagglehub.dataset_download("daniellopez01/credit-risk")

#print("Path to dataset files:", path) 

In [47]:
import pandas as pd

import numpy as np


### Exercícios:
---

1. **Agrupamento por `credit_history` com resumo estatístico:**  
   Agrupe os dados pela coluna `credit_history` e calcule:
   - A soma total dos valores da coluna `amount`;
   - A idade média (`age`) para cada grupo.


In [48]:
import pandas as pd
df = pd.read_csv('credit.csv')

df.head()


,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes


In [49]:
resultado = df.groupby(['credit_history']).agg(
    total_quantidade=('amount', 'sum'),  
    media_idade=('age', 'mean')     
).reset_index()

resultado


,credit_history,total_quantidade,media_idade
0,critical,904795,38.436860
1,good,1611708,33.877358
2,perfect,212227,34.300000
3,poor,378629,36.136364
4,very good,163899,36.265306



2. **Criação de uma coluna com média ajustada:**  
   Crie uma nova coluna chamada `amount_ratio`, que contém o valor de `amount` dividido pela média de `amount` em cada categoria de `purpose`.


In [50]:
df_grupo = df.groupby('purpose') 

df['amount_ratio'] = df['amount'] / df_grupo['amount'].transform('mean')

df_amountratio =df[['purpose', 'amount', 'amount_ratio']]

df_amountratio.head(5)


,purpose,amount,amount_ratio
0,furniture/appliances,1169,0.435506
1,furniture/appliances,5951,2.217018
2,education,2096,0.727979
3,furniture/appliances,7882,2.936403
4,car,4870,1.292397



3. **Classificação de idade:**  
   Crie uma nova coluna chamada `age_group` que classifica os valores de `age` em categorias:
   - "Under 30" se a idade for menor que 30;
   - "30-50" se a idade estiver entre 30 e 50;
   - "Over 50" se a idade for maior que 50.

In [51]:

def classificar_idade(age):
    if age < 30:
        return "Under 30", 1
    elif 30 <= age <= 50:
        return "30-50", 2
    elif age > 50:
        return "Over 50", 3
    else:
        return "Unknown", 0


In [52]:
df[['grupo_idade', 'grupo_idade_id']] = df['age'].apply(lambda x: pd.Series(classificar_idade(x)))


df_grupo_idade = df[['age', 'grupo_idade', 'grupo_idade_id']]

df_grupo_idade.head()

   age grupo_idade  grupo_idade_id
0   67     Over 50               3
1   22    Under 30               1
2   49       30-50               2
3   45       30-50               2
4   53     Over 50               3



4. **Análise de variabilidade por `employment_duration` e `housing`:**  
   Agrupe os dados pelas colunas `employment_duration` e `housing` e calcule:
   - A média e o desvio padrão (`std`) da coluna `amount`.


In [62]:
variabilidade = df.groupby(['employment_duration', 'housing']).agg(
    mean_amount=('amount', 'mean'), 
    std_amount=('amount', 'std')
).reset_index()


variabilidade.head(5)

,employment_duration,housing,mean_amount,std_amount
0,1 - 4 years,other,5112.961538,3554.197468
1,1 - 4 years,own,2999.162698,2590.607900
2,1 - 4 years,rent,2799.114754,2339.122571
3,4 - 7 years,other,4788.454545,3254.082770
4,4 - 7 years,own,3556.053435,3101.828623



5. **Distribuição de idade mínima e máxima por `job`:**  
   Agrupe os dados pela coluna `job` e calcule:
   - A idade mínima (`age_min`) e a idade máxima (`age_max`) para cada categoria de emprego.


In [57]:
idade_distribuicao = df.groupby('job').agg(
    age_min=('age', 'min'), 
    age_max=('age', 'max')   
).reset_index()

idade_distribuicao

,job,age_min,age_max
0,management,20,75
1,skilled,19,68
2,unemployed,23,74
3,unskilled,19,74



6. **Diferença entre o valor do empréstimo e a média por histórico de crédito (`transform`):**  
   Crie uma nova coluna chamada `amount_diff`, que contém a diferença entre o valor de `amount` e a média de `amount` dentro de cada grupo de `credit_history`.


In [59]:
df['amount_diff'] = df['amount'] - df.groupby('credit_history')['amount'].transform('mean')

df_eprestimo_por_credito = df[['credit_history', 'amount', 'amount_diff']]

df_eprestimo_por_credito.head(5)



,credit_history,amount,amount_diff
0,critical,1169,-1919.037543
1,good,5951,2910.041509
2,critical,2096,-992.037543
3,good,7882,4841.041509
4,poor,4870,567.397727



7. **Quantidade de dependentes por tipo de conta bancária (`groupby` e `agg`):**  
   Agrupe os dados pela coluna `checking_balance` e calcule:
   - A soma total de `dependents`;
   - O número médio de dependentes por categoria de conta bancária.



In [60]:
dependetes_por_conta = df.groupby('checking_balance').agg(
    total_dependents=('dependents', 'sum'),  
    mean_dependents=('dependents', 'mean')  
).reset_index()

dependetes_por_conta


,checking_balance,total_dependents,mean_dependents
0,1 - 200 DM,300,1.115242
1,< 0 DM,326,1.189781
2,> 200 DM,72,1.142857
3,unknown,457,1.159898
